In [28]:
import warnings
import scanpy as sc
import squidpy as sq
import anndata as an
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from urllib import request
import json
import os

sc.settings.set_figure_params(dpi=80)
#sc.set_figure_params(facecolor="white", figsize=(8, 8))
warnings.simplefilter(action='ignore', category=FutureWarning)
sc.settings.verbosity = 3

## Global variables

In [36]:
dir_path = '../../results/'

In [25]:
de_dir_path = '../../results/DiffExpr/young_vs_old/'

In [37]:
os.listdir(de_dir_path)

['edgeR', 'scanpy']

### DE results

In [13]:
# Load EdgeR results
edgeR_DE = dict()

for file in os.listdir(de_dir_path + 'edgeR'):
    layer = file.split('_')[0]
    df = pd.read_csv(de_dir_path + 'edgeR/' + file, index_col=0)
    edgeR_DE[layer] = df

edgeR_DE.keys()

dict_keys(['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'WM'])

In [14]:
edgeR_DE['L1']

,logFC,logCPM,F,PValue,FDR
NLRP1,-12.290117,7.616894,8.849669e+02,4.300965e-52,5.082880e-48
DPYSL3,3.618613,8.494822,6.061350e+02,1.164502e-45,6.881043e-42
VXN,-11.139452,6.644319,5.611696e+02,2.859678e-43,1.126522e-39
YBX1,3.958872,8.784563,5.298523e+02,4.782240e-43,1.412913e-39
TUBA1A,3.208877,11.739539,4.999432e+02,6.182635e-42,1.461328e-38
...,...,...,...,...,...
HUS1,0.000509,3.249055,1.254909e-06,9.991083e-01,9.992944e-01
TUBGCP2,0.013668,5.847461,9.082083e-03,9.991178e-01,9.992944e-01
UBE2G2,0.000170,5.152333,9.525497e-07,9.992231e-01,9.992944e-01
PGS1,0.059686,4.460213,7.521595e-02,9.992876e-01,9.992944e-01


### Load adata

In [43]:
# read young human adata object
human_young = sc.read_h5ad(dir_path + 'filtered_samples/concated/human.h5ad')
print(human_young)

# add _young sufix to layer name
human_young.obs.label.replace({"L6a": "L6", "L6b": "L6"}, inplace=True)
human_young.obs.label = human_young.obs.label.astype('str') + '_young'

AnnData object with n_obs × n_vars = 15417 × 19966
    obs: 'in_tissue', 'array_row', 'array_col', 'label', 'sample_id', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_hb', 'pct_counts_hb', 'total_counts_ribo', 'pct_counts_ribo'
    var: 'gene_ids', 'feature_types', 'mt', 'hb', 'ribo'
    uns: 'qc_good_spots_colors', 'qc_n_genes_by_counts_colors', 'qc_pct_counts_ribo_colors', 'qc_total_counts_colors', 'spatial'
    obsm: 'spatial'


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [46]:
# read young human adata object
human_old = sc.read_h5ad(dir_path + 'filtered_samples/concated/spatial_libd_human.h5ad')
print(human_old)

# add _young sufix to layer name
human_old.obs.label = human_old.obs.label.astype('str') + '_old'

AnnData object with n_obs × n_vars = 42588 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'label', 'sample_id', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_hb', 'pct_counts_hb', 'total_counts_ribo', 'pct_counts_ribo'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'hb', 'ribo'
    uns: 'qc_good_spots_colors', 'qc_n_genes_by_counts_colors', 'qc_pct_counts_mt_colors', 'qc_pct_counts_ribo_colors', 'qc_total_counts_colors', 'spatial'
    obsm: 'spatial'


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [47]:
# concat old and young files
adata = an.concat([human_young, human_old], merge='same',uns_merge="unique")
adata.obs_names_make_unique()
adata

C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 58005 × 19404
    obs: 'in_tissue', 'array_row', 'array_col', 'label', 'sample_id', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_hb', 'pct_counts_hb', 'total_counts_ribo', 'pct_counts_ribo'
    var: 'feature_types', 'mt', 'hb', 'ribo'
    uns: 'qc_good_spots_colors', 'qc_n_genes_by_counts_colors', 'qc_total_counts_colors', 'spatial', 'qc_pct_counts_mt_colors'
    obsm: 'spatial'

In [48]:
samples = list(adata.uns['spatial'].keys())
samples

['human_759',
 'human_j12',
 'human_j3',
 'human_j4',
 'human_j6',
 '151507',
 '151508',
 '151509',
 '151510',
 '151669',
 '151670',
 '151671',
 '151672',
 '151673',
 '151674',
 '151675',
 '151676']

In [49]:
del human_young, human_old